<a href="https://colab.research.google.com/github/Olowojaye/AI-for-Robotics/blob/main/Lesson_2_Problem_Set_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# The function localize takes the following arguments:
#
# colors:
#        2D list, each entry either 'R' (for red cell) or 'G' (for green cell)
#
# measurements:
#        list of measurements taken by the robot, each entry either 'R' or 'G'
#
# motions:
#        list of actions taken by the robot, each entry of the form [dy,dx],
#        where dx refers to the change in the x-direction (positive meaning
#        movement to the right) and dy refers to the change in the y-direction
#        (positive meaning movement downward)
#        NOTE: the *first* coordinate is change in y; the *second* coordinate is
#              change in x
#
# sensor_right:
#        float between 0 and 1, giving the probability that any given
#        measurement is correct; the probability that the measurement is
#        incorrect is 1-sensor_right
#
# p_move:
#        float between 0 and 1, giving the probability that any given movement
#        command takes place; the probability that the movement command fails
#        (and the robot remains still) is 1-p_move; the robot will NOT overshoot
#        its destination in this exercise
#
# The function should RETURN (not just show or print) a 2D list (of the same
# dimensions as colors) that gives the probabilities that the robot occupies
# each cell in the world.
#
# Compute the probabilities by assuming the robot initially has a uniform
# probability of being in any cell.
#
# Also assume that at each step, the robot:
# 1) first makes a movement,
# 2) then takes a measurement.
#
# Motion:
#  [0,0] - stay
#  [0,1] - right
#  [0,-1] - left
#  [1,0] - down
#  [-1,0] - up

def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    # >>> Insert your code here <<<
    def move(p, motion, p_move):
      # Effect motion in x-direction
      q = [[0 for i in range(len(p[0]))] for i in range(len(p))]
      for i in range(len(p)):
        for j in range(len(p[i])):
          s =  p_move * p[i][(j-motion[1]) % len(p[i])] + (1-p_move) * p[i][j]
          q[i][j] = s
      # Effect motion in y-direction
      r = [[0 for i in range(len(p[0]))] for i in range(len(p))]
      for i in range(len(p[1])):
        for j in range(len(p)):
          t = p_move * q[(j-motion[0]) % len(p)][i] + (1-p_move) * q[j][i]
          r[j][i] = t
      return r
    
    def sense(p, colors, measurement, p_right):
      q=[[0 for i in range(len(p[0]))] for i in range(len(p))]
      for i in range(len(p)):
        for j in range(len(p[i])):
          right = (measurement == colors[i][j])
          q[i][j] = p[i][j] * (right * p_right + (1-right) * (1-p_right))
      s = sum([sum(i) for i in q])
      for i in range(len(q)):
        for j in range(len(q[i])):
          q[i][j] = q[i][j] / s
      return q
    
    for i in range(len(motions)):
      p = move(p, motions[i], p_move)
      p = sense(p, colors, measurements[i], sensor_right)
    return p

def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print('[' + ',\n '.join(rows) + ']')
    
#############################################################
# For the following test case, your output should be 
# [[0.01105, 0.02464, 0.06799, 0.04472, 0.02465],
#  [0.00715, 0.01017, 0.08696, 0.07988, 0.00935],
#  [0.00739, 0.00894, 0.11272, 0.35350, 0.04065],
#  [0.00910, 0.00715, 0.01434, 0.04313, 0.03642]]
# (within a tolerance of +/- 0.001 for each entry)

colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)
show(p) # displays your answer


[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]


In [36]:
# To test for sense function independently

def sense(p, colors, measurement, p_right):
  q=[[0 for i in range(len(p[0]))] for i in range(len(p))]
  for i in range(len(p)):
    for j in range(len(p[i])):
      right = (measurement == colors[i][j])
      q[i][j] = p[i][j] * (right * p_right + (1-right) * (1-p_right))
  s = sum([sum(i) for i in q])
  for i in range(len(q)):
    for j in range(len(q[i])):
      q[i][j] = q[i][j] / s
  return q
print(sense(p, colors, measurements[0], p_right = 0.7))

[[0.005942555223584472, 0.030892316688607816, 0.08524910400705357, 0.024031160966948614, 0.013245536737861904], [0.003843494668582577, 0.005465164715415639, 0.10903143849995182, 0.04292270596196567, 0.005024216230771144], [0.003974685953270195, 0.004805564089625112, 0.14133202850643542, 0.44320101504532544, 0.021844639555488304], [0.004893024946083526, 0.003843494668582577, 0.007709993364514887, 0.023175988148244134, 0.019571872021687062]]


In [35]:
# To test for move function independently

def move(p, motion, p_move):
  # Effect motion in x-direction
  q = [[0 for i in range(len(p[0]))] for i in range(len(p))]
  for i in range(len(p)):
    for j in range(len(p[i])):
      s =  p_move * p[i][(j-motion[1]) % len(p[i])] + (1-p_move) * p[i][j]
      q[i][j] = s
  # Effect motion in y-direction
  r = [[0 for i in range(len(p[0]))] for i in range(len(p))]
  for i in range(len(p[1])):
    for j in range(len(p)):
      t = p_move * q[(j-motion[0]) % len(p)][i] + (1-p_move) * q[j][i]
      r[j][i] = t
  return r
print(move([[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]], motions[2], p_move=0.8))

[[0.0, 0.0, 0.8], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.19999999999999996]]
